In [ ]:
import duckdb
import pandas as pd
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql


In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%sql duckdb:///duckdb.db

In [ ]:
%%sql
ATTACH '../data/raw/igem_parts_registry.sqlite' (TYPE sqlite);

In [ ]:
%%sql
SELECT * FROM duckdb_tables();

In [ ]:
%%sql
df_parts << SELECT * FROM igem_parts_registry.parts;

In [ ]:
df_parts = df_parts.set_index("part_id", drop=False)
condition1 = df_parts.part_type == 'Composite'
condition2 = df_parts.status == 'Available'
df_composite = df_parts[condition1 & condition2]
df_composite

In [ ]:
# Step 1: Split the strings to get lists of nodes
df_parts['nodes'] = df_parts.deep_u_list.apply(lambda x: x.strip('_').split('_'))

# Create the edge list by pairing the identifier with each node in the list
edge_list = [(str(row['part_id']), str(node)) for _, row in df_parts.iterrows() for node in row['nodes']]

df_edge_list_parts = pd.DataFrame(edge_list, columns=["source", "target"])
df_edge_list_parts["relation"] = "composed_by"

In [ ]:
# Initialize a undirected graph
G = nx.from_pandas_edgelist(df_edge_list_parts, source='source', target='target', edge_attr=["relation"])#, create_using=nx.DiGraph())

In [ ]:
outfile = Path("../data/interim/parts_network.graphml")
outfile.parent.mkdir(exist_ok=True, parents=True)
nx.write_graphml(G, outfile)

In [ ]:
df_edge_list_parts[df_edge_list_parts.target == '57575']

In [ ]:
df_edge_list_parts[df_edge_list_parts.source == 57575]

In [ ]:
df_edge_list_parts[df_edge_list_parts.target == '7249']

In [ ]:
df_edge_list_parts[df_edge_list_parts.target == 7249]